In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sweetviz as sv
import pickle
from itertools import zip_longest
from tabulate import tabulate
import pdfkit
import seaborn as sns
from itertools import zip_longest
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, cross_val_score  
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.dummy import DummyClassifier
from sklearn import metrics
import sklearn.linear_model as lm
from sklearn.linear_model import LogisticRegression
import sklearn.tree as tree
from sklearn.metrics import accuracy_score
import sklearn.neighbors as neighbors
import sklearn.naive_bayes as nb
import sklearn.svm as svm
import sklearn.ensemble as ensemble
import xgboost
from sklearn.metrics import confusion_matrix
from yellowbrick.classifier import ConfusionMatrix
from sklearn.neighbors import KNeighborsClassifier
from yellowbrick.features import (JointPlotVisualizer,)
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost

# Creacion de los datos 

In [38]:
X2 = pd.DataFrame({
    "a": range(5),
    "b": [-100, -50, 0, 200, 1000],
})
print(X2)

   a     b
0  0  -100
1  1   -50
2  2     0
3  3   200
4  4  1000


# Estandarizar

Algunos algoritmos, como SVM, funcionan mejor cuando los datos
está estandarizado. Cada columna debe tener un valor medio de 0.
y desviación estándar de 1. Sklearn proporciona un .fit_transform
método que combina .fit y .transform:

El código utiliza la clase `StandardScaler` de la biblioteca `scikit-learn` (`sklearn`) para realizar el escalado estándar (también conocido como normalización z-score) en los datos contenidos en el DataFrame `X2`. Aquí tienes una explicación en español:

**Creación de la instancia del StandardScaler:**
- `std = preprocessing.StandardScaler()` crea un objeto `StandardScaler` que se utilizará para realizar el escalado estándar.

**Ajuste y transformación de los datos:**
- `std.fit_transform(X2)` ajusta el escalador a los datos en `X2` y luego transforma esos datos, es decir, realiza el escalado estándar. El resultado escalado se almacena en `X2_scaled`.

Después de este proceso, `X2_scaled` contendrá los datos de `X2` escalados de acuerdo con la media y la desviación estándar de cada columna, lo que significa que los datos tendrán una media de cero y una desviación estándar de uno. Esto es útil en muchas aplicaciones de aprendizaje automático donde se benefician de tener características con escalas comparables.


In [39]:
from sklearn import preprocessing
std = preprocessing.StandardScaler()
std.fit_transform(X2)

array([[-1.41421356, -0.75995002],
       [-0.70710678, -0.63737744],
       [ 0.        , -0.51480485],
       [ 0.70710678, -0.02451452],
       [ 1.41421356,  1.93664683]])

`std.scale_` no es un atributo directo de la clase `StandardScaler` en scikit-learn. En cambio, puedes acceder a la desviación estándar utilizada para el escalado estándar a través del atributo `std.scale_` después de ajustar el escalador a los datos.

Aquí hay un ejemplo de cómo puedes acceder a la desviación estándar después de ajustar el `StandardScaler`:

In [40]:
std.scale_

array([  1.41421356, 407.92156109])

In [41]:
std.mean_

array([  2., 210.])

In [42]:
std.var_

array([2.000e+00, 1.664e+05])

*Estandarizamos toda las columnas solo con los valores de la segunda*

In [43]:
X_std = (X2 - X2.mean()) / X2.std()
X_std

,a,b
0,-1.264911,-0.679720
1,-0.632456,-0.570088
2,0.000000,-0.460455
3,0.632456,-0.021926
4,1.264911,1.732190


In [44]:
X_std.mean()

a    4.440892e-17
b    0.000000e+00
dtype: float64

In [45]:
X_std.std()

a    1.0
b    1.0
dtype: float64

# Scale to Range

*Escalar al rango es traducir datos para que estén entre 0 y 1, incluido. Tener los datos vinculados puede resultar útil. Sin embargo, si Si tienes valores atípicos, probablemente quieras tener cuidado al usar esto:*

In [46]:
from sklearn import preprocessing
mms = preprocessing.MinMaxScaler()
mms.fit(X2)
mms.transform(X2)

array([[0.        , 0.        ],
       [0.25      , 0.04545455],
       [0.5       , 0.09090909],
       [0.75      , 0.27272727],
       [1.        , 1.        ]])

*Para la version de pandas:*

In [47]:
(X2 - X2.min()) / (X2.max() - X2.min())

,a,b
0,0.00,0.000000
1,0.25,0.045455
2,0.50,0.090909
3,0.75,0.272727
4,1.00,1.000000


# "Dummy" Variables

*Podemos usar pandas para crear variables ficticias a partir de categorías fecha. Esto también se conoce como codificación one-hot o indicador. codificación. Las variables ficticias son especialmente útiles si los datos son nominal (desordenado). La función get_dummies en pandas crea varias columnas para una columna categórica, cada una con un 1 o 0 si la columna original tenía ese valor:*

In [48]:
X_cat = pd.DataFrame({"name": ["George", "Paul"],"inst": ["Bass", "Guitar"]})
print(X_cat)

     name    inst
0  George    Bass
1    Paul  Guitar



Aquí está la versión pandas. Tenga en cuenta que la opción drop_first puede ser
Se utiliza para eliminar una columna (una de las columnas ficticias es una
combinación lineal de las otras columnas):

In [49]:
pd.get_dummies(X_cat, drop_first=True)

,name_Paul,inst_Guitar
0,False,False
1,True,True


In [50]:
import janitor as jn

X_cat2 = pd.DataFrame({
    "A": [1, None, 3],
    "names": ["Fred,George", "George", "John,Paul"],
})

jn.expand_column(X_cat2, "names", sep=",")


,A,names,Fred,George,John,Paul
0,1.0,"Fred,George",1,1,0,0
1,NaN,George,0,1,0,0
2,3.0,"John,Paul",0,0,1,1


# Codificador de etiquetas

Una alternativa a la variable de codificación ficticia es la codificación de etiquetas.
Esto tomará datos categóricos y asignará a cada valor un número.
Es útil para datos de alta cardinalidad. Este codificador impone ordinalidad,
que puede ser deseable o no. Puede ocupar menos
espacio que la codificación one-hot, y algunos algoritmos (de árbol) pueden
lidiar con esta codificación.
El codificador de etiquetas solo puede procesar una columna a la vez:

In [51]:
from sklearn import preprocessing
import pandas as pd

X_cat = pd.DataFrame({"name": ["George", "Paul"],"inst": ["Bass", "Guitar"]})
print(X_cat)

lab = preprocessing.LabelEncoder()

# Apply LabelEncoder to each column
X_cat["name"] = lab.fit_transform(X_cat["name"])
X_cat["inst"] = lab.fit_transform(X_cat["inst"])

print(X_cat)



     name    inst
0  George    Bass
1    Paul  Guitar
   name  inst
0     0     0
1     1     1


*Este código creará una nueva serie de datos numéricos de un panda serie. Usamos el método .as_ordered para asegurar que el La categoría está ordenada:*

In [52]:
lab.inverse_transform([1, 1, 0])

array(['Guitar', 'Guitar', 'Bass'], dtype=object)

También puedes usar pandas para etiquetar la codificación. Primero, conviertes el
columna a un tipo de columna categórica y luego extraiga la
código numérico del mismo.
Este código creará una nueva serie de datos numéricos de un panda.
serie. Usamos el método .as_ordered para asegurar que el
La categoría está ordenada:

In [53]:
X_cat.name.astype("category").cat.as_ordered().cat.codes +1

0    1
1    2
dtype: int8

# Frequency Encoding

Otra opción para manejar datos categóricos de alta cardinalidad es
para codificarlo en frecuencia. Esto significa reemplazar el nombre del
categoría con el recuento que tenía en los datos de entrenamiento. Usaremos
pandas para hacer esto. Primero, usaremos pandas .value_counts
método para hacer un mapeo (una serie de pandas que mapea cadenas
para contar). Con el mapeo podemos usar el método .map para
de la codificación:

In [54]:
mapping = X_cat.name.value_counts()
print(mapping)

name
0    1
1    1
Name: count, dtype: int64


# Pulling Categories from Strings

Una forma de aumentar la precisión del modelo del Titanic es tirar
sacar títulos de los nombres. Un truco rápido para encontrar los más comunes.
triples es usar la clase Counter:

Usando estas manipulaciones y pandas, puedes crear muñecos
variables o combinar columnas con recuentos bajos en otras categorías
(o dejarlos)

In [55]:
from collections import Counter

# Importacion de los datos del titanic 
# Ruta completa al archivo CSV
file_path = 'C:/Users/felip/OneDrive/Documentos/Python/Codigos/Ciencia_Datos/Datos_titanic_CSV/titanic3.csv'
# Importar la base de datos
df = pd.read_csv(file_path)
X= df
c = Counter()
def triples(val):
    for i in range(len(val)):
        c[val[i : i + 3]] += 1

La función apply en pandas se utiliza para aplicar una función a lo largo de un eje del DataFrame

In [56]:
X.name.apply(triples)

0       None
1       None
2       None
3       None
4       None
        ... 
1304    None
1305    None
1306    None
1307    None
1308    None
Name: name, Length: 1309, dtype: object

In [57]:
c.most_common(10)

[(', M', 1282),
 (' Mr', 954),
 ('r. ', 830),
 ('Mr.', 757),
 ('s. ', 460),
 ('n, ', 320),
 (' Mi', 283),
 ('iss', 261),
 ('ss.', 261),
 ('Mis', 260)]

In [58]:
df.name.str.extract("([A-Za-z]+)\.", expand=False).head()

<>:1: DeprecationWarning: invalid escape sequence '\.'
<>:1: DeprecationWarning: invalid escape sequence '\.'
C:\Users\felip\AppData\Local\Temp\ipykernel_19608\1228772012.py:1: DeprecationWarning: invalid escape sequence '\.'
  df.name.str.extract("([A-Za-z]+)\.", expand=False).head()


0      Miss
1    Master
2      Miss
3        Mr
4       Mrs
Name: name, dtype: object

In [59]:
df.name.str.extract("([A-Za-z]+)\.", expand=False).value_counts()

<>:1: DeprecationWarning: invalid escape sequence '\.'
<>:1: DeprecationWarning: invalid escape sequence '\.'
C:\Users\felip\AppData\Local\Temp\ipykernel_19608\2743877583.py:1: DeprecationWarning: invalid escape sequence '\.'
  df.name.str.extract("([A-Za-z]+)\.", expand=False).value_counts()


name
Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Mlle          2
Ms            2
Major         2
Capt          1
Sir           1
Dona          1
Jonkheer      1
Countess      1
Don           1
Mme           1
Lady          1
Name: count, dtype: int64

# Other Categorical Encoding

La biblioteca categorical_encoding es un conjunto de transformadores scikit-learn Se utiliza para convertir datos categóricos en datos numéricos. A Una característica interesante de esta biblioteca es que admite la salida de pandas. DataFrames (a diferencia de scikit-learn, que los transforma en numerosas matrices).Un algoritmo implementado en la biblioteca es un codificador hash.Esto es útil si no sabes cuántas categorías tienes.con anticipación o si estás usando una bolsa de palabras para representar texto. Esto dividirá las columnas categóricas en n_components. Si está utilizando el aprendizaje en línea (modelos que se pueden actualizar),Esto puede ser muy útil:

In [60]:
import category_encoders as ce
X_cat = pd.DataFrame({"name": ["George", "Paul"],"inst": ["Bass", "Guitar"]})
he = ce.HashingEncoder(verbose=1)
he.fit_transform(X_cat)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
0,0,0,0,1,0,1,0,0
1,0,2,0,0,0,0,0,0


In [61]:
size_df = pd.DataFrame(
    {
        "name": ["Fred", "John", "Matt"], 
        "size": ["small", "med", "xxl"],   
    }
)
print(size_df)


   name   size
0  Fred  small
1  John    med
2  Matt    xxl


In [62]:
import category_encoders as ce  # Importa la biblioteca category_encoders y la aliasa como ce

# Crea un objeto OrdinalEncoder llamado ore
ore = ce.OrdinalEncoder(
    mapping=[
        {
            "col": "size",  # Especifica la columna a codificar
            "mapping": {
                "small": 1,  # Mapea la categoría "small" al valor 1
                "med": 2,    # Mapea la categoría "med" al valor 2
                "lg": 3,     # Mapea la categoría "lg" al valor 3
            },
        }
    ]
)


In [63]:
ore.fit_transform(size_df)

,name,size
0,Fred,1.0
1,John,2.0
2,Matt,-1.0


Esta referencia explica muchos de los algoritmos del categórico_
biblioteca de codificación.
Si tiene datos de alta cardinalidad (una gran cantidad de valores únicos)
considere usar uno de los codificadores bayesianos que generan un
una sola columna por columna categórica. Estos son TargetEn
codificador, LeaveOneOutEncoder, WOEEncoder, JamesSteinEncoder,
y MEestimateEncoder.
Por ejemplo, para convertir la columna de supervivencia del Titanic en una mezcla
de la probabilidad posterior del objetivo y la probabilidad previa
Dada la información del título (categórica), utilice el siguiente código:

In [67]:
import category_encoders as ce
import pandas as pd

def get_title(df):
    return df.name.str.extract("([A-Za-z]+)\\.", expand=False)

# Supongamos que df es tu DataFrame
df = pd.DataFrame({
    "name": ["Mr. Smith", "Miss Johnson", "Dr. Brown"],
    "survived": [1, 0, 1]
})

te = ce.TargetEncoder(cols="Title")
result = te.fit_transform(df.assign(Title=get_title), df.survived)["Title"].head()

print(result)


0    0.710036
1    0.579928
2    0.710036
Name: Title, dtype: float64


# Date Feature Engineering

La biblioteca fastai tiene una función add_datepart que generará
columnas de atributos de fecha basadas en una columna de fecha y hora. Esto es
útil ya que la mayoría de los algoritmos de aprendizaje automático no podrían
inferir este tipo de señal a partir de una representación numérica de un
fecha:

In [79]:
from fastai.tabular.all import add_datepart
import pandas as pd

# Crear un DataFrame de ejemplo con una columna de fecha
df = pd.DataFrame({
    "fecha": pd.to_datetime(["2022-01-01", "2022-02-01", "2022-03-01"])
})

# Aplicar add_datepart para agregar características de la fecha
add_datepart(df, "fecha")

# Mostrar el DataFrame resultante
print(df)

   fechaYear  fechaMonth  fechaWeek  fechaDay  fechaDayofweek  fechaDayofyear  \
0       2022           1         52         1               5               1   
1       2022           2          5         1               1              32   
2       2022           3          9         1               1              60   

   fechaIs_month_end  fechaIs_month_start  fechaIs_quarter_end  \
0              False                 True                False   
1              False                 True                False   
2              False                 True                False   

   fechaIs_quarter_start  fechaIs_year_end  fechaIs_year_start  fechaElapsed  
0                   True             False                True  1.640995e+09  
1                  False             False               False  1.643674e+09  
2                  False             False               False  1.646093e+09  


In [80]:
df.T

,0,1,2
fechaYear,2022,2022,2022
fechaMonth,1,2,3
fechaWeek,52,5,9
fechaDay,1,1,1
fechaDayofweek,5,1,1
fechaDayofyear,1,32,60
fechaIs_month_end,False,False,False
fechaIs_month_start,True,True,True
fechaIs_quarter_end,False,False,False
fechaIs_quarter_start,True,False,False


# Add col_na Feature

La biblioteca fastai solía tener una función para crear una columna.
para completar un valor faltante (con la mediana) e indicar que un
faltaba valor. Puede haber alguna señal al saber que
faltaba un valor. Aquí hay una copia de la función y una
ejemplo usándolo

In [81]:
from pandas.api.types import is_numeric_dtype

In [82]:
def fix_missing(df, col, name, na_dict):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (
            name in na_dict
        ):
            df[name + "_na"] = pd.isnull(col)
            filler = (
                na_dict[name]
                if name in na_dict
                else col.median()
            )
            df[name] = col.fillna(filler)
            na_dict[name] = filler
            
    return na_dict


In [83]:
data = pd.DataFrame({"A": [0, None, 5, 100]})

In [84]:
fix_missing(data, data.A, "A", {})

{'A': 5.0}

In [85]:
data

,A,A_na
0,0.0,False
1,5.0,True
2,5.0,False
3,100.0,False


In [86]:
data = pd.DataFrame({"A": [0, None, 5, 100]})
data["A_na"] = data.A.isnull()
data["A"] = data.A.fillna(data.A.median())

In [87]:
data

,A,A_na
0,0.0,False
1,5.0,True
2,5.0,False
3,100.0,False


# Manual Feature Engineering